# SageMaker Models using the Apache MXNet Module API


### Import Ipython package
* IPython.display - Public API for display tools in IPython.

In [1]:
from IPython.display import HTML

### Importing amazon packages
* boto3 - The AWS SDK for Python to write software that uses Amazon services like S3 and EC2.
* sagemaker - Python SDK for training and deploying machine learning models on Amazon SageMaker.
* sagemaker.mxnet - the Amazon sagemaker custom Apache MXNet code.
* get_execution_role - Return the role ARN whose credentials are used to call the API.

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.mxnet import MXNet

### The training script

Your python script should implement a few methods like train, model_fn, transform_fn, input_fn etc. SagaMaker will call appropriate method when needed. https://docs.aws.amazon.com/sagemaker/latest/dg/mxnet-training-inference-code-template.html

In [3]:
!cat mnist.py

import logging

import gzip
import mxnet as mx
import numpy as np
import os
import struct


def load_data(path):
    with gzip.open(find_file(path, "labels.gz")) as flbl:
        struct.unpack(">II", flbl.read(8))
        labels = np.fromstring(flbl.read(), dtype=np.int8)
        
    with gzip.open(find_file(path, "images.gz")) as fimg:
        _, _, rows, cols = struct.unpack(">IIII", fimg.read(16))
        images = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(labels), rows, cols)
        images = images.reshape(images.shape[0], 1, 28, 28).astype(np.float32) / 255
        
    return labels, images


def find_file(root_path, file_name):
    for root, dirs, files in os.walk(root_path):
        if file_name in files:
            return os.path.join(root, file_name)


def build_graph():
    data = mx.sym.var('data')
    data = mx.sym.flatten(data=data)
    
    fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
    act1 = mx.sym.Activation(data=fc1, act_type="relu")
    
 

### Set up for model training
* Set the bucket location to save your custom code.
* Set the bucket location to save trained model.
* Get the path to train and test data.
* Get the role ARN whose credentials are used to call the API to instantiate the estimator.
* Get the region where the model willl be hosted

In [4]:
region = boto3.Session().region_name
region

'us-east-2'

In [5]:
custom_code_upload_location = 's3://loonybucket/sagemaker/customcode/mxnet'

In [6]:
model_artifacts_location = 's3://loonybucket/sagemaker/artifacts'

In [7]:
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

In [8]:
role = get_execution_role()
role

'arn:aws:iam::324118574079:role/service-role/AmazonSageMaker-ExecutionRole-20180209T192191'

### Training the model
* Instantiate an estimator object and pass in the code as the entry point parameter.
* Train and deploy the model

In [9]:
mnist_estimator = MXNet(entry_point='mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.m4.xlarge',
                        hyperparameters={'learning_rate': 0.1})

In [10]:
mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

NoSuchBucket: An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist

In [43]:
predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-py2-cpu-2018-03-09-02-01-05-526
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-py2-cpu-2018-03-09-02-01-05-526


------------------------------------------------------------------------------------------------!

## Validating the model
* Invoke the html script to read in an input. The pixel data from your drawing will be loaded into a data variable in this notebook.
* Using the predictor object to classify the handwritten digit.
* Raw predictions and Labelled predictions display the probabilities of the digit being each of the defined labels.
* Most likely answer prints the label with the maximum probability. 

In [76]:
HTML(open("input.html").read())

In [77]:
print(data)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 

In [78]:
type(data)

list

In [79]:
len(data)

1

In [80]:
print(len(data[0][0]))
print(len(data[0][1]))
print(len(data[0][2]))

print(len(data[0][27]))

28
28
28
28


In [81]:
response = predictor.predict(data)
print('Raw prediction result:')
response

Raw prediction result:


[[1.1966670779363126e-15,
  2.680095789742154e-08,
  5.847177741991061e-10,
  0.9975250363349915,
  6.280418845235922e-18,
  0.0023851492442190647,
  6.983133375951136e-16,
  3.804462966883193e-09,
  8.982335566543043e-05,
  1.455597642419093e-09]]

In [82]:
labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
labeled_predictions

Labeled predictions: 


[(0, 1.1966670779363126e-15),
 (1, 2.680095789742154e-08),
 (2, 5.847177741991061e-10),
 (3, 0.9975250363349915),
 (4, 6.280418845235922e-18),
 (5, 0.0023851492442190647),
 (6, 6.983133375951136e-16),
 (7, 3.804462966883193e-09),
 (8, 8.982335566543043e-05),
 (9, 1.455597642419093e-09)]

In [83]:
labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

Most likely answer: (3, 0.9975250363349915)


## Delete the prediction endpoint


In [84]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-mxnet-py2-cpu-2018-03-09-02-01-05-526
